In [1]:
import sys
import pandas as pd

sys.path.append('../src') 
from util import get_rcode_df
from util import read_csv
from es_client import *

In [2]:
rcode_df = get_rcode_df('../data/road_code.csv')

In [3]:
es = ESClient() 
df = es._search_by_aptname('real-estate', "단대푸르지오") 
print (df) 

     date    거래금액  건축년도     년  법정동     아파트   월      일   전용면적   지번   지역코드   층
0  201209  47,500  2012  2012  단대동  단대푸르지오   9   1~10  84.70  100  41131   9
1  201209  33,200  2012  2012  단대동  단대푸르지오   9   1~10  59.72  100  41131   1
2  201209  32,300  2012  2012  단대동  단대푸르지오   9  11~20  59.97  100  41131   1
3  201209  44,800  2012  2012  단대동  단대푸르지오   9  21~30  84.70  100  41131   2
4  201210  47,500  2012  2012  단대동  단대푸르지오  10   1~10  84.70  100  41131  15
5  201305  44,800  2012  2013  단대동  단대푸르지오   5   1~10  84.74  100  41131   2
6  201306  43,900  2012  2013  단대동  단대푸르지오   6  11~20  84.70  100  41131   1
7  201307  43,900  2012  2013  단대동  단대푸르지오   7   1~10  84.70  100  41131   1
8  201309  43,500  2012  2013  단대동  단대푸르지오   9   1~10  84.74  100  41131   2
9  201309  44,800  2012  2013  단대동  단대푸르지오   9  21~30  84.74  100  41131   2


# 거래 데이터 + 메타 데이터 조인

* 지역코드의 데이터 타입은 (int)로 자동으로 읽어옴 (pandas에서 dtype을 추론해서 가져옴)
* rcode_df의 지역코드는 index로 이미 `util.get_rcode_df`에서 처리했음
* 두개의 index를 기준으로 join

In [4]:
df.set_index('지역코드').dtypes

date      int64
거래금액     object
건축년도      int64
년         int64
법정동      object
아파트      object
월         int64
일        object
전용면적    float64
지번       object
층         int64
dtype: object

In [5]:
df['지역코드'] = df['지역코드'].astype(str)

In [6]:
rcode_df.index = rcode_df.index.map(str)

In [7]:
df.set_index('지역코드').join(rcode_df)

,date,거래금액,건축년도,년,법정동,아파트,월,일,전용면적,지번,층,0,1,2
41131,201209,"47,500",2012,2012,단대동,단대푸르지오,9,1~10,84.70,100,9,경기도,성남시 수정구,창업로57번길
41131,201209,"33,200",2012,2012,단대동,단대푸르지오,9,1~10,59.72,100,1,경기도,성남시 수정구,창업로57번길
41131,201209,"32,300",2012,2012,단대동,단대푸르지오,9,11~20,59.97,100,1,경기도,성남시 수정구,창업로57번길
41131,201209,"44,800",2012,2012,단대동,단대푸르지오,9,21~30,84.70,100,2,경기도,성남시 수정구,창업로57번길
41131,201210,"47,500",2012,2012,단대동,단대푸르지오,10,1~10,84.70,100,15,경기도,성남시 수정구,창업로57번길
41131,201305,"44,800",2012,2013,단대동,단대푸르지오,5,1~10,84.74,100,2,경기도,성남시 수정구,창업로57번길
41131,201306,"43,900",2012,2013,단대동,단대푸르지오,6,11~20,84.70,100,1,경기도,성남시 수정구,창업로57번길
41131,201307,"43,900",2012,2013,단대동,단대푸르지오,7,1~10,84.70,100,1,경기도,성남시 수정구,창업로57번길
41131,201309,"43,500",2012,2013,단대동,단대푸르지오,9,1~10,84.74,100,2,경기도,성남시 수정구,창업로57번길
41131,201309,"44,800",2012,2013,단대동,단대푸르지오,9,21~30,84.74,100,2,경기도,성남시 수정구,창업로57번길


# 전처리에 join_meta 를 추가

* join 하기전에 데이터 타입 변환
* 컬럼명 시/군/도로명/시군구를 추가

In [8]:
from preprocessing import join_meta

In [9]:
join_meta(df, rcode_df)

,date,거래금액,건축년도,년,법정동,아파트,월,일,전용면적,지번,지역코드,층,시,구,도로명,시군구
0,201209,"47,500",2012,2012,단대동,단대푸르지오,9,1~10,84.70,100,41131,9,경기도,성남시 수정구,창업로57번길,경기도 성남시 수정구
1,201209,"33,200",2012,2012,단대동,단대푸르지오,9,1~10,59.72,100,41131,1,경기도,성남시 수정구,창업로57번길,경기도 성남시 수정구
2,201209,"32,300",2012,2012,단대동,단대푸르지오,9,11~20,59.97,100,41131,1,경기도,성남시 수정구,창업로57번길,경기도 성남시 수정구
3,201209,"44,800",2012,2012,단대동,단대푸르지오,9,21~30,84.70,100,41131,2,경기도,성남시 수정구,창업로57번길,경기도 성남시 수정구
4,201210,"47,500",2012,2012,단대동,단대푸르지오,10,1~10,84.70,100,41131,15,경기도,성남시 수정구,창업로57번길,경기도 성남시 수정구
5,201305,"44,800",2012,2013,단대동,단대푸르지오,5,1~10,84.74,100,41131,2,경기도,성남시 수정구,창업로57번길,경기도 성남시 수정구
6,201306,"43,900",2012,2013,단대동,단대푸르지오,6,11~20,84.70,100,41131,1,경기도,성남시 수정구,창업로57번길,경기도 성남시 수정구
7,201307,"43,900",2012,2013,단대동,단대푸르지오,7,1~10,84.70,100,41131,1,경기도,성남시 수정구,창업로57번길,경기도 성남시 수정구
8,201309,"43,500",2012,2013,단대동,단대푸르지오,9,1~10,84.74,100,41131,2,경기도,성남시 수정구,창업로57번길,경기도 성남시 수정구
9,201309,"44,800",2012,2013,단대동,단대푸르지오,9,21~30,84.74,100,41131,2,경기도,성남시 수정구,창업로57번길,경기도 성남시 수정구
